In [0]:
! wget http://model.scir.yunfutech.com/model/ltp_data_v3.4.0.zip

--2020-05-21 10:00:11--  http://model.scir.yunfutech.com/model/ltp_data_v3.4.0.zip
Resolving model.scir.yunfutech.com (model.scir.yunfutech.com)... 47.246.18.229, 47.246.18.230, 47.246.18.231, ...
Connecting to model.scir.yunfutech.com (model.scir.yunfutech.com)|47.246.18.229|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 637993278 (608M) [application/zip]
Saving to: ‘ltp_data_v3.4.0.zip’

ltp_data_v3.4.0.zip 100%[===================>] 608.44M  6.42MB/s    in 93s     

2020-05-21 10:01:45 (6.53 MB/s) - ‘ltp_data_v3.4.0.zip’ saved [637993278/637993278]



In [0]:
!unzip ltp_data_v3.4.0.zip

Archive:  ltp_data_v3.4.0.zip
   creating: ltp_data_v3.4.0/
  inflating: ltp_data_v3.4.0/cws.model  
  inflating: ltp_data_v3.4.0/md5.txt  
  inflating: ltp_data_v3.4.0/ner.model  
  inflating: ltp_data_v3.4.0/parser.model  
  inflating: ltp_data_v3.4.0/pisrl.model  
  inflating: ltp_data_v3.4.0/pos.model  
  inflating: ltp_data_v3.4.0/version  


In [0]:
!tar -zxvf AutoMaster_TrainSet.tgz

._AutoMaster_TrainSet.csv
AutoMaster_TrainSet.csv


In [0]:
!pip install pyltp

     |████████████████████████████████| 5.3MB 2.8MB/s 
     |████████████████████████████████| 5.3MB 2.8MB/s 
  Created wheel for pyltp: filename=pyltp-0.2.1-cp36-cp36m-linux_x86_64.whl size=31990973 sha256=8359f04e8ea78f16538f599396108735c31fbd5484bd58c3d62ebf787e1c7fd4
  Stored in directory: /root/.cache/pip/wheels/fc/3a/35/b11293efb2c77c0e7b6fa574271d51cddd9abd1f634535343c
  Created wheel for pyltp: filename=pyltp-0.2.1-cp36-cp36m-linux_x86_64.whl size=31990973 sha256=8359f04e8ea78f16538f599396108735c31fbd5484bd58c3d62ebf787e1c7fd4
  Stored in directory: /root/.cache/pip/wheels/fc/3a/35/b11293efb2c77c0e7b6fa574271d51cddd9abd1f634535343c
Successfully built pyltp
Successfully built pyltp


In [0]:
import tensorflow as tf
import numpy as np
from pyltp import SentenceSplitter,NamedEntityRecognizer,Postagger,Parser,Segmentor
import pandas as pd
from gensim.models.word2vec import LineSentence, Word2Vec
import time, shutil, os
print(tf.__version__)
import matplotlib.pyplot as plt
# shutil.rmtree('checkpoint')

2.2.0


In [0]:
vocabulary_dimension = 100
unites = 128
batch_size = 32

In [0]:
df = pd.read_csv('./AutoMaster_TrainSet.csv')

In [0]:
cws_model = "./ltp_data_v3.4.0/cws.model"

def get_word_list(sentence=None,sentences=None,model=cws_model):
    #得到分词
    segmentor = Segmentor()
    segmentor.load(model)
    if sentences is not None:
        for i, s in enumerate(sentences):
            sentences[i] = list(segmentor.segment(s))
        return sentences
    else:
        word_list = list(segmentor.segment(sentence))
    segmentor.release()
    return word_list

In [0]:
inp = list(df.Question)
outp = list(df.Report)
assert len(inp) == len(outp)

# 去除随时联系, 查阅资料等无意义回答
drop_index_set = set()
for index, (i, o) in enumerate(zip(inp, outp)):
    if not type(i) == type(o) == str or len(i) < 8 or len(o) < 8:
        drop_index_set.add(index)
inp = [el for i, el in enumerate(inp) if i not in drop_index_set]
outp = [el for i, el in enumerate(outp) if i not in drop_index_set]

inp = get_word_list(sentences=inp)
outp = get_word_list(sentences=outp)

In [0]:
def get_length(tensor):
    length = sorted([len(t) for t in tensor])
    return length[int(len(length) * 0.95)]

inp_length = get_length(inp)
outp_length = get_length(outp)

with open('../gensim_input.txt', 'w') as f:
    for s in inp:
#         s = s[:inp_length]
#         s = s + ['<pad>'] * (inp_length - len(s))
        s = ['<start>'] + s + ['<end>']
        f.write(' '.join(s) + '\n')
    for s in outp:
#         s = s[:outp_length]
#         s = s + ['<pad>'] * (outp_length - len(s))
        s = ['<start>'] + s + ['<end>']
        f.write(' '.join(s) + '\n')

In [0]:
! head 10 ../gensim_input.txt

head: cannot open '10' for reading: No such file or directory
==> ../gensim_input.txt <==
<start> 2010 款 宝马 X1 ， 2011年 出厂 ， 2.0 排量 ， 通用 6L45 变速箱 ， 原地 换 挡 位 PRND 车辆 闯动 ， 行驶 升降档 正常 ， 4 轮 离 地 换 挡 无 冲击感 ， 更换 变速箱 油 12L 无 改变 。 试过 一 辆 2014年 进口 X1 原地 换 挡位 也 有 冲击感 ， 这 是 什么 情况 ， 哪里 的 问题 <end>
<start> 3.0V6 发动机 号 在 什么 位置 ， 有 照片 最 好 ！ <end>
<start> 2012款 奔驰 c180 怎么样 ， 维修 保养 ， 动力 ， 值得 拥有 吗 <end>
<start> 科鲁兹 变速箱 旁边 漏 机油 <end>
<start> 我 要 怎么 才 能 知道 车子 断开 电瓶 电源 之后 要 不 要 做 节气门 或 防盗 ， 音响 等 的 重置 ， 还有 节气门 重 置 是 不 是 所有 车型 都 一样 ， 打开 电源 不 启动 车子 ， 油门 踩 到底 五 秒 ， 重复 几 次 就 可以 了 。 其他 设备 要 密码 才 行 是 不 是 <end>
<start> 昌河 Q35 音响 怎么 拆 装 <end>
<start> 长安 35 朝阳 轮胎 不 要 里面 的 钢圈 。 用 我 自己 的 钢圈 多少 钱 外面 换 多少 钱 <end>
<start> 吉利 远景 外 球笼 上 那个 大 螺丝 是 顺时针 拧 下来 还是 反 的 <end>
<start> 五菱 之 光 6376nf ， 自己 装 了 铁将军 防盗器 ， 五线马达 的 白褐 两 根 信号线 接到 什么 地方 去 ？ 我 想 实现 一 控 三马达 <end>
<start> 丰田 花冠 行驶 了 十万 公里 皮带 要 换 吗 <end>


In [0]:
model = Word2Vec(corpus_file='../gensim_input.txt', size=vocabulary_dimension, min_count=5, workers=8, sg=0, negative=5, iter=5)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
class Encoder(tf.keras.Model):
    def __init__(self, vocabulary_number, vocabulary_dimension, vocabulary_matrix, gru_unites, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.gru_unites = gru_unites
        self.embedding = tf.keras.layers.Embedding(vocabulary_number, vocabulary_dimension, 
                                               weights=[vocabulary_matrix], trainable=False
                                                  )
        self.gru = tf.keras.layers.GRU(gru_unites, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
    
    def call(self, inp, hidden):
        x = self.embedding(inp)
        x, hidden = self.gru(x, initial_state=hidden)
        return x, hidden
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_size, self.gru_unites))
    

In [0]:
class Decoder(tf.keras.Model):
    def __init__(self, vocabulary_number, vocabulary_dimension, vocabulary_matrix, unites, batch_size):
        super(Decoder, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocabulary_number, vocabulary_dimension, 
                                                  weights=[vocabulary_matrix], trainable=False
                                                   )
        self.gru = tf.keras.layers.GRU(unites, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocabulary_number)
        
        # BahdanauAttention
        self.w1 = tf.keras.layers.Dense(unites)
        self.w2 = tf.keras.layers.Dense(unites)
        self.v = tf.keras.layers.Dense(1)
    
    def call(self, x, dec_hidden, enc_output):
        # dec_hidden shape == (batch_size, hidden size)
        # enc_output shape == (batch_size, max_length, hidden_size)

        hidden_with_time_axis = tf.expand_dims(dec_hidden, 1)
        score = tf.nn.tanh(self.w1(enc_output) + self.w2(hidden_with_time_axis))
        attention_weights = tf.nn.softmax(self.v(score), axis=1)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
#         print(x.shape)
#         print(x)
#         e = tf.keras.layers.Embedding(len(targ_lang.word2idx), vocabulary_dimension)
#         print(e(x))
#         self.embedding(1)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        output, state = self.gru(x)
        
        # output shape == (batch_size * max_length, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * max_length, vocab)
        output = self.fc(output)
        
        return output, state, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))


In [0]:
class LanguageIndex():
    def __init__(self, lang, model):
        self.lang = lang
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set(model.wv.vocab.keys())
        
        self.create_index()
        
    def create_index(self):
        self.word2idx['<pad>'] = 0
        self.word2idx['<start>'] = 1
        self.word2idx['<end>'] = 2
        self.word2idx['<unknown>'] = 3
        for index, word in enumerate(self.vocab):
            if word not in self.word2idx:
                self.word2idx[word] = len(self.word2idx)
        
        for word, index in self.word2idx.items():
            self.idx2word[index] = word

In [0]:
inp_lang = LanguageIndex(inp, model)
targ_lang = LanguageIndex(outp, model)

In [0]:
inp = [[inp_lang.word2idx[w] if w in inp_lang.word2idx else inp_lang.word2idx['<unknown>'] for w in s] for s in inp]
outp = [[targ_lang.word2idx[w] if w in targ_lang.word2idx else targ_lang.word2idx['<unknown>'] for w in s] + [targ_lang.word2idx['<end>']] for s in outp]

In [0]:
pad_index = targ_lang.word2idx['<pad>']
def loss_function(real, pred):  #(128,) (128, 13706)
    mask = 1 - np.equal(real, pad_index)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    return tf.reduce_mean(loss_)

In [0]:
max_length_inp, max_length_tar = get_length(inp), get_length(outp)

input_tensor = tf.keras.preprocessing.sequence.pad_sequences(inp, 
                                                                 maxlen=max_length_inp,
                                                                 padding='post')
target_tensor = tf.keras.preprocessing.sequence.pad_sequences(outp, 
                                                                  maxlen=max_length_tar, 
                                                                  padding='post')
max_length_inp, max_length_tar, input_tensor.shape, target_tensor.shape

input_tensor_backup = input_tensor
target_tensor_backup = target_tensor

In [0]:
input_tensor = input_tensor_backup
target_tensor = target_tensor_backup

valid_input_tensor = input_tensor[-1000:]
valid_target_tensor = target_tensor[-1000:]
input_tensor = input_tensor[:-1000]
target_tensor = target_tensor[:-1000]

BUFFER_SIZE = len(input_tensor)
dataset = tf.data.Dataset.from_tensor_slices((input_tensor, target_tensor)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(batch_size, drop_remainder=True)

BUFFER_SIZE = len(valid_input_tensor)
valid_dataset = tf.data.Dataset.from_tensor_slices((valid_input_tensor, valid_target_tensor)).shuffle(BUFFER_SIZE)
# valid_dataset = dataset.batch(1, drop_remainder=True)

input_tensor.shape, target_tensor.shape, valid_input_tensor.shape, valid_target_tensor.shape

((77662, 84), (77662, 53), (1000, 84), (1000, 53))

In [0]:
w = '<pad>'
model.wv[w] = np.zeros((100,))
model.wv[w]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [0]:
w = '<unknown>'
model.wv[w] = np.zeros((100,))

In [0]:
inp_vocabulary_matrix = np.random.rand(len(inp_lang.word2idx), vocabulary_dimension)
targ_vocabulary_matrix = np.random.rand(len(targ_lang.word2idx), vocabulary_dimension)
for i, w in inp_lang.idx2word.items():
#     if w not in model.wv:
#         continue
    inp_vocabulary_matrix[i] = model.wv[w]
for i, w in targ_lang.idx2word.items():
#     if w not in model.wv:
#         continue
    targ_vocabulary_matrix[i] = model.wv[w]

In [0]:
w = '<unknown>'
model.wv[w] = np.average(inp_vocabulary_matrix, axis=0)
model.wv[w]
inp_vocabulary_matrix[inp_lang.word2idx[w]] = model.wv[w]
targ_vocabulary_matrix[targ_lang.word2idx[w]] = model.wv[w]
targ_vocabulary_matrix[targ_lang.word2idx['<unknown>']]

array([ 0.0101694 ,  0.02259252,  0.11632625,  0.02447463,  0.02669374,
       -0.08202558, -0.03130912,  0.08206645, -0.04524239,  0.06280863,
       -0.04385983, -0.13220068, -0.06803403,  0.0245816 ,  0.0033028 ,
       -0.00821976, -0.01338859,  0.04783527,  0.01554555, -0.02564134,
       -0.0395499 ,  0.07473961, -0.00889863, -0.0308418 ,  0.00665309,
       -0.04806956,  0.05889331, -0.0671872 , -0.1843942 ,  0.00806049,
       -0.1114675 , -0.04841806,  0.10744677,  0.01303872,  0.03537503,
       -0.03466585,  0.03079063,  0.06901155, -0.09861822, -0.00430365,
       -0.08244321, -0.0120569 ,  0.05148925,  0.02119892, -0.00774634,
       -0.10662674, -0.10767667, -0.04720252, -0.0724934 ,  0.06154174,
       -0.04347353, -0.17784323,  0.07147583,  0.10679646,  0.18406751,
       -0.00500955, -0.09562746,  0.06395808,  0.07865652, -0.09576591,
       -0.01956278,  0.00245131,  0.10670749,  0.09028936,  0.00246607,
        0.04912075,  0.05779324, -0.06859382,  0.09628783, -0.03

In [0]:
encoder = Encoder(len(inp_lang.word2idx), vocabulary_dimension, inp_vocabulary_matrix, unites, batch_size)
decoder = Decoder(len(targ_lang.word2idx), vocabulary_dimension, targ_vocabulary_matrix, unites, batch_size)

In [0]:
optimizer = tf.keras.optimizers.Adam(0.01)

In [0]:
checkpoint_dir = '/content/drive/My Drive/Colab Notebooks/checkpoint'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

['checkpoint', 'ckpt-1.data-00000-of-00002', 'ckpt-1.data-00001-of-00002', 'ckpt-2.data-00000-of-00002', 'ckpt-1.index', 'ckpt-3.data-00000-of-00002', 'ckpt-2.data-00001-of-00002', 'ckpt-4.data-00000-of-00002', 'ckpt-2.index', 'ckpt-3.data-00001-of-00002', 'ckpt-5.data-00000-of-00002', 'ckpt-3.index', 'ckpt-4.data-00001-of-00002', 'ckpt-4.index', 'ckpt-5.data-00001-of-00002', 'ckpt-5.index']


In [0]:
EPOCHS = 10
loss_plot = []
t1 = time.time()
for epoch in range(EPOCHS):
    t2 = time.time()
    total_loss = 0
    hidden = encoder.initialize_hidden_state()
    
    for (batch, (inp, targ)) in enumerate(dataset):
        
        loss = 0
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)
            
            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * batch_size, 1)

            for t in range(1, targ.shape[1]):
                predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_output)
                del attention_weights

                loss += loss_function(targ[:, t], predictions)

                dec_input = tf.expand_dims(targ[:, t], 1)
                
        total_loss += (loss / targ.shape[1])
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables)
        optimizer.apply_gradients(zip(gradients, variables))

    checkpoint.save(file_prefix=checkpoint_prefix)

    print(f'loss: {total_loss/(batch + 1)}, time cost: {time.time() - t2}')
    loss_plot.append(total_loss/(batch + 1))
print(f'finished training, time cost: {time.time() - t1}')
plt.plot(loss_plot)

loss: 1.8955897092819214, time cost: 2396.579042196274
loss: 1.7520654201507568, time cost: 2391.679209470749


KeyboardInterrupt: ignored

In [0]:
optimizer = tf.keras.optimizers.Adam()

In [0]:
EPOCHS = 100
loss_plot = []
t1 = time.time()
for epoch in range(EPOCHS):
    t2 = time.time()
    total_loss = 0
    hidden = encoder.initialize_hidden_state()
    
    for (batch, (inp, targ)) in enumerate(dataset):

        print(f'\r{batch}/{len(input_tensor) // batch_size}', end='')
        loss = 0
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)
            
            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * batch_size, 1)

            for t in range(1, targ.shape[1]):
                predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_output)
                del attention_weights

                loss += loss_function(targ[:, t], predictions)

                dec_input = tf.expand_dims(targ[:, t], 1)
                
        total_loss += (loss / targ.shape[1])
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables)
        optimizer.apply_gradients(zip(gradients, variables))

    checkpoint.save(file_prefix=checkpoint_prefix)

    print(f'loss: {total_loss/(batch + 1)}, time cost: {time.time() - t2}')
    loss_plot.append(total_loss/(batch + 1))
print(f'finished training, time cost: {time.time() - t1}')
plt.plot(loss_plot)
raise

2425/2426loss: 1.669633388519287, time cost: 2370.304208278656
2425/2426loss: 1.6319600343704224, time cost: 2374.7030160427094
2425/2426loss: 1.6134244203567505, time cost: 2380.693681240082
213/2426

KeyboardInterrupt: ignored

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
EPOCHS = 100
loss_plot = []
t1 = time.time()
for epoch in range(EPOCHS):
    t2 = time.time()
    total_loss = 0
    hidden = encoder.initialize_hidden_state()
    
    for (batch, (inp, targ)) in enumerate(dataset):

        print(f'\r{batch}/{len(input_tensor) // batch_size}', end='')
        loss = 0
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)
            
            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * batch_size, 1)

            for t in range(1, targ.shape[1]):
                predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_output)
                del attention_weights

                loss += loss_function(targ[:, t], predictions)

                dec_input = tf.expand_dims(targ[:, t], 1)
                
        total_loss += (loss / targ.shape[1])
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables)
        optimizer.apply_gradients(zip(gradients, variables))

    checkpoint.save(file_prefix=checkpoint_prefix)

    print(f'loss: {total_loss/(batch + 1)}, time cost: {time.time() - t2}')
    loss_plot.append(total_loss/(batch + 1))
print(f'finished training, time cost: {time.time() - t1}')
plt.plot(loss_plot)
raise

8/2426

KeyboardInterrupt: ignored

In [0]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir)).assert_consumed()

In [0]:
def id2word(sentence, lang):
    if type(sentence) != list:
        sentence = list(sentence.numpy())

    sentence = [lang.idx2word[el] for el in sentence if el != lang.word2idx['<pad>']]
    return ''.join(sentence)


In [63]:
BUFFER_SIZE = len(valid_input_tensor)
valid_dataset = tf.data.Dataset.from_tensor_slices((valid_input_tensor, valid_target_tensor))

valid_input_tensor.shape, valid_target_tensor.shape

((1000, 84), (1000, 53))

In [68]:
hidden = tf.zeros((1, unites))
    
for (batch, (inp, targ)) in enumerate(valid_dataset):
    print('input:', id2word(inp, inp_lang))
    print('targ:', id2word(targ, targ_lang))
    inp = tf.expand_dims(inp, 0)
    enc_output, enc_hidden = encoder(inp, hidden)

    dec_hidden = enc_hidden
    
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 1)


    output = []
    for _ in range(100):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_output)
        predictions = (tf.keras.layers.Softmax()(tf.reshape(predictions, (-1,))))
        predictions = tf.argmax(predictions)


        predictions = int(predictions.numpy())
        output.append(predictions)
        if predictions == targ_lang.word2idx['<end>']:
          break
        

        dec_input = tf.expand_dims([predictions], 1)
    print('predict:', id2word(output, targ_lang))            

    print('#' * 100)
    


input: 一三年宝马X6今天早晨打火两秒就熄火怎么回事？踩油门打火也不行。遥控开关门都正常一键启动的。
targ: 建议到4s先编程升级软件，如故障依旧旧，要检查油压和进气门积碳。<end>
predict: ，根据描述情况分析，根据你的描述，这个情况应该是车辆的问题，需要检查一下，检查一下，检查一下，检查一下，检查一下，检查一下，检查一下，检查一下，检查一下，检查一下，检查一下，检查一下，检查一下，检查一下，检查一下，检查一下，检查一下，检查一下，检查一下，检查一下，检查一下，检查一下，检查一下，检查一下，检查一下，检查一下，检查一下
####################################################################################################
input: 前几天换变速箱油，感觉有点不彻底，变速箱换油需要拆开换变速箱底壳，换<unknown>吗？
targ: 这款变速箱是没有独立的油底壳的，只有重力放油了，七升换的还是比较可以的，不需要<unknown><end>
predict: ，根据你的描述1.这个情况，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱油，变速箱
####################################################################################################
input: 08年帕萨特领驭，低速加油门特别是转弯时，驾驶室内左侧前方保险<unknown>会发出连续的嗒嗒嗒声，声音不像是机械摩擦或者撞击声，而是<unknown>。松油门声音消失。P档或者空档加油门时无故障声音。尝试换过火花塞，点火线圈，故障照旧。
targ: 你好，这种情况下，建议你先拔了电子真空泵插头是否还存在这种声音。<end>
predict: ，根据描述情况分析，检查一下，检查一下，检查一下，检查一下，检查一下，检查一下，检查一下，检查一下，检查一

KeyboardInterrupt: ignored